In [1]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

In [2]:
number_cores = 8
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [33]:
!dir /users/trush/CSC496/FrequentItemsets/data

frequent.csv  links.csv  movies.csv  ratings.csv  README.txt  tags.csv


In [4]:
ratings = sc.textFile("/users/trush/CSC496/FrequentItemsets/data/ratings.csv")

In [5]:
ratings.take(5)

['userId,movieId,rating,timestamp',
 '1,1,4.0,964982703',
 '1,3,4.0,964981247',
 '1,6,4.0,964982224',
 '1,47,5.0,964983815']

In [6]:
ratings.cache()

/users/trush/CSC496/FrequentItemsets/data/ratings.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
%%time
ratings.count()

CPU times: user 13.7 ms, sys: 2.98 ms, total: 16.7 ms
Wall time: 1.13 s


100837

In [8]:
%%time
ratings.count()

CPU times: user 12.1 ms, sys: 2.5 ms, total: 14.6 ms
Wall time: 184 ms


100837

In [9]:
%%time
ratings.count()

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 180 ms


100837

### Questions

- What movies are often reviewed together (hence watched together ...)

In [10]:
ratings.getNumPartitions()

2

In [11]:
ratingHeader = ratings.first()
print(ratingHeader)

userId,movieId,rating,timestamp


In [12]:
ratingsOnly = ratings.filter(lambda x: x != ratingHeader)
ratingsOnly.take(5)

['1,1,4.0,964982703',
 '1,3,4.0,964981247',
 '1,6,4.0,964982224',
 '1,47,5.0,964983815',
 '1,50,5.0,964982931']

In [13]:
baskets = ratingsOnly.map(lambda x: x.split(',')) \
    .map(lambda x: (int(x[0]),int(x[1]))) \
    .groupByKey().map(lambda x: list(set(x[1])))
baskets.take(5)

[[115713,
  122882,
  48516,
  91529,
  80906,
  91658,
  114060,
  131724,
  77455,
  79132,
  106782,
  1704,
  112552,
  99114,
  89774,
  109487,
  68157,
  318,
  58559,
  86345,
  333,
  3578,
  60756,
  74458,
  6874,
  8798,
  80489,
  71535,
  46970],
 [1025,
  3079,
  3591,
  2571,
  3083,
  21,
  1046,
  2583,
  4121,
  538,
  539,
  2076,
  2078,
  32,
  1057,
  4641,
  1060,
  2599,
  553,
  1580,
  45,
  47,
  4144,
  1073,
  52,
  1077,
  1079,
  1080,
  58,
  1084,
  1597,
  1086,
  2109,
  2628,
  1094,
  4166,
  588,
  1103,
  593,
  1617,
  595,
  599,
  3160,
  608,
  2145,
  2150,
  3175,
  3176,
  1641,
  106,
  1136,
  2683,
  125,
  126,
  2174,
  4226,
  2692,
  3204,
  4741,
  648,
  2186,
  4239,
  4246,
  2712,
  1179,
  2203,
  2204,
  4252,
  1183,
  4765,
  162,
  1188,
  4260,
  1704,
  171,
  1196,
  1197,
  1198,
  1199,
  176,
  4273,
  1203,
  1719,
  3255,
  1211,
  1213,
  190,
  1219,
  708,
  1732,
  1733,
  1734,
  1225,
  2762,
  2763,
  3788,


In [14]:
numPartitions = baskets.getNumPartitions()
print(numPartitions)

2


We can't use standard mapping to items, because that means on single basket at a time. To refer to the idea of **local** itemsets, we will use [mapPartitions](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.mapPartitions.html)

In [15]:
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

In [16]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9], 3)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[6, 15, 24]

In [17]:
# set support level
support = 100

In [18]:
import sys
import numpy as np
import time
from itertools import combinations

In [19]:
def apriori(iterator):
    baskets = list(iterator)
    support_part = np.floor(float(support)/numPartitions)
    k = 2
    d = {}
    frequent_items = []
    frequent_itemset = []
    
    for basket in baskets:
        for item in basket:
            if item not in d:
                d[item] = 1
            else:
                d[item] = d[item] + 1
  
    # Pass 1
    for item in d:
        if d[item] >= support_part:
            frequent_items.append(item)
    
    frequent_itemset = [(i,1) for i in frequent_items]
    frequent_items = [{i} for i in frequent_items]
    
    # Pass 2 and onward
    while (len(frequent_items) > 0):
        # we build candidate_set from frequent_items set
        candidate_set = get_candidate_set(frequent_items,k)
        # reset frequent_items set because now it should contain larget sets. 
        frequent_items = []
        for candidate in candidate_set:
            c = 0
            for basket in baskets:
                if candidate.issubset(basket):
                    c = c + 1
            if c >= support_part:
                frequent_items.append(candidate)
        if len(frequent_items) > 0:
            # why 1?
            frequent_itemset = frequent_itemset + [(tuple(i),1) for i in frequent_items]
        k = k + 1        
    return(frequent_itemset)

In [20]:
def get_candidate_set(frequent_items,k):
    cadidate_set = []
    if k == 2:
        # why enumerate but not standard loop?
        for i,e1 in enumerate(frequent_items):
            for j,e2 in enumerate(frequent_items):
                if j > i:
                    # you can merge two dictionaries in Python
                    pair = e1|e2
                    if len(pair) == k and pair not in cadidate_set:
                        cadidate_set.append(pair)
    else:
        for i,e1 in enumerate(frequent_items):
            for j,e2 in enumerate(frequent_items):
                if j > i:
                    common = e1.intersection(e2)
                    if len(common) == k-2:
                        pair = e1|e2
                        if pair not in cadidate_set:
                            pairs = list(combinations(pair,k-1))
                            c = 0
                            for p in pairs:
                                if c == k-2:
                                    break
                                if common.issubset(p) == False:
                                    if set(p) in frequent_items:
                                        c = c + 1
                            if c == k-2:
                                cadidate_set.append(pair)
    return cadidate_set

In [21]:
def count_occurances(iterator,candidate_itemset):
    baskets = list(iterator)
    candidate_local = []
    for candidate in candidate_itemset:
        c = 0
        if type(candidate) != tuple:
            candidate = (candidate,)
        for basket in baskets:
            flag =  all(item in basket  for item in candidate)
            if flag == True:
                c = c + 1
        candidate_local.append((candidate,c))
    
    return(candidate_local)

In [22]:
m = 'milk'
c = 'coke'
b = 'beer'
j = 'juice'
p = 'pepsi'
numPartitions = 1
baskets_example = [[m,c,b],[m,p,j],[m,b],[c,j],[m,p,b],[m,c,b,j],[c,b,j],[b,c]]
# set support level
support = 2
fi = apriori(baskets_example)
print(fi)

[('milk', 1), ('coke', 1), ('beer', 1), ('pepsi', 1), ('juice', 1), (('milk', 'coke'), 1), (('milk', 'beer'), 1), (('milk', 'pepsi'), 1), (('juice', 'milk'), 1), (('coke', 'beer'), 1), (('juice', 'coke'), 1), (('juice', 'beer'), 1), (('milk', 'coke', 'beer'), 1), (('juice', 'coke', 'beer'), 1)]


In [23]:
# set support level
support = 120
numPartitions = baskets.getNumPartitions()
print(numPartitions)

2


In [24]:
#MapReduce Phase 1
candidate_itemset = baskets.mapPartitions(apriori).groupByKey().map(lambda x:x[0]).collect()

In [25]:
frequent_items = baskets.mapPartitions(lambda x: count_occurances(x,candidate_itemset)) \
    .reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] >= support).collect()

In [26]:
frequent_items

[((608, 318), 128),
 ((1196, 318), 133),
 ((1198, 318), 124),
 ((2762, 318), 123),
 ((260, 318), 156),
 ((296, 318), 222),
 ((2858, 318), 134),
 ((50, 318), 163),
 ((592, 318), 127),
 ((110, 318), 167),
 ((150, 318), 151),
 ((780, 318), 124),
 ((356, 318), 231),
 ((380, 318), 124),
 ((480, 318), 154),
 ((5952, 318), 129),
 ((858, 318), 135),
 ((1210, 318), 123),
 ((3578, 356), 126),
 ((2571, 47), 132),
 ((593, 2571), 160),
 ((2571, 2959), 180),
 ((2571, 527), 134),
 ((2571, 589), 144),
 ((4993, 2571), 157),
 ((7153, 2571), 148),
 ((1, 2571), 123),
 ((32, 296), 131),
 ((32, 356), 120),
 ((1580, 260), 120),
 ((356, 1580), 134),
 ((593, 47), 150),
 ((589, 47), 126),
 ((296, 588), 130),
 ((356, 588), 141),
 ((364, 588), 127),
 ((480, 588), 130),
 ((593, 2959), 137),
 ((593, 457), 140),
 ((593, 527), 148),
 ((593, 589), 148),
 ((593, 377), 121),
 ((593, 1), 121),
 ((608, 260), 126),
 ((608, 296), 144),
 ((608, 356), 124),
 ((648, 780), 129),
 ((1196, 1198), 148),
 ((1196, 260), 190),
 ((296

In [29]:
max_len = 0
for item in frequent_items:
    if len(item[0]) > max_len:
        max_len = len(item[0])

d = {i:[] for i in range(1,max_len+1)}
for item in frequent_items:
    d[len(item[0])].append(tuple(sorted(item[0])))

output_file="/users/trush/CSC496/FrequentItemsets/data/frequent.csv"
with open(output_file,'w') as file_write:
    for key in d:
        d[key].sort()
        file_write.write(str(d[key][1:-1]).replace(',)',')'))
        file_write.write('\n\n')
file_write.close()

## FP-Growth

- [Spark libraries](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html)
- [Mining frequent patterns without candidate generation](http://staff.cs.psu.ac.th/sirirut/courses/344-463/weka/fp-growthpaper.pdf)


In [30]:
basketsFP = ratingsOnly.map(lambda x: x.split(',')) \
    .map(lambda x: (int(x[0]),int(x[1]))) \
    .groupByKey().map(lambda x: (x[0],list(set(x[1]))))
basketsFP.take(5)

[(2,
  [115713,
   122882,
   48516,
   91529,
   80906,
   91658,
   114060,
   131724,
   77455,
   79132,
   106782,
   1704,
   112552,
   99114,
   89774,
   109487,
   68157,
   318,
   58559,
   86345,
   333,
   3578,
   60756,
   74458,
   6874,
   8798,
   80489,
   71535,
   46970]),
 (4,
  [1025,
   3079,
   3591,
   2571,
   3083,
   21,
   1046,
   2583,
   4121,
   538,
   539,
   2076,
   2078,
   32,
   1057,
   4641,
   1060,
   2599,
   553,
   1580,
   45,
   47,
   4144,
   1073,
   52,
   1077,
   1079,
   1080,
   58,
   1084,
   1597,
   1086,
   2109,
   2628,
   1094,
   4166,
   588,
   1103,
   593,
   1617,
   595,
   599,
   3160,
   608,
   2145,
   2150,
   3175,
   3176,
   1641,
   106,
   1136,
   2683,
   125,
   126,
   2174,
   4226,
   2692,
   3204,
   4741,
   648,
   2186,
   4239,
   4246,
   2712,
   1179,
   2203,
   2204,
   4252,
   1183,
   4765,
   162,
   1188,
   4260,
   1704,
   171,
   1196,
   1197,
   1198,
   1199,
   176,
   427

In [31]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession

spark = SparkSession(sc)
df_FP = spark.createDataFrame(basketsFP, ["id","items"])
df_FP.take(5)

[Row(id=2, items=[115713, 122882, 48516, 91529, 80906, 91658, 114060, 131724, 77455, 79132, 106782, 1704, 112552, 99114, 89774, 109487, 68157, 318, 58559, 86345, 333, 3578, 60756, 74458, 6874, 8798, 80489, 71535, 46970]),
 Row(id=4, items=[1025, 3079, 3591, 2571, 3083, 21, 1046, 2583, 4121, 538, 539, 2076, 2078, 32, 1057, 4641, 1060, 2599, 553, 1580, 45, 47, 4144, 1073, 52, 1077, 1079, 1080, 58, 1084, 1597, 1086, 2109, 2628, 1094, 4166, 588, 1103, 593, 1617, 595, 599, 3160, 608, 2145, 2150, 3175, 3176, 1641, 106, 1136, 2683, 125, 126, 2174, 4226, 2692, 3204, 4741, 648, 2186, 4239, 4246, 2712, 1179, 2203, 2204, 4252, 1183, 4765, 162, 1188, 4260, 1704, 171, 1196, 1197, 1198, 1199, 176, 4273, 1203, 1719, 3255, 1211, 1213, 190, 1219, 708, 1732, 1733, 1734, 1225, 2762, 2763, 3788, 2770, 4308, 215, 222, 3809, 1250, 2791, 232, 2282, 235, 1259, 1265, 1266, 3317, 759, 247, 4347, 1279, 1282, 1283, 260, 1288, 265, 1291, 3851, 4881, 2324, 1304, 2843, 4381, 3358, 800, 2336, 4896, 3365, 4902, 296, 1

In [32]:
minSupport = 120 / df_FP.count()
print(minSupport)
fpGrowth = FPGrowth(itemsCol="items", minSupport=minSupport, minConfidence=0.6)
model = fpGrowth.fit(df_FP)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df_FP).show()

0.19672131147540983
+----------------+----+
|           items|freq|
+----------------+----+
|          [1206]| 120|
|           [364]| 172|
|      [364, 296]| 123|
|      [364, 588]| 127|
|      [364, 480]| 128|
| [364, 480, 356]| 120|
|      [364, 356]| 144|
|         [79132]| 143|
|          [1240]| 131|
|          [1193]| 133|
|           [589]| 224|
|      [589, 296]| 165|
| [589, 296, 318]| 126|
| [589, 296, 356]| 143|
|      [589, 260]| 137|
|[589, 260, 2571]| 120|
|     [589, 2571]| 144|
|      [589, 110]| 156|
| [589, 110, 296]| 122|
| [589, 110, 480]| 127|
+----------------+----+
only showing top 20 rows

+---------------+----------+------------------+------------------+
|     antecedent|consequent|        confidence|              lift|
+---------------+----------+------------------+------------------+
|[480, 296, 318]|     [356]|0.9477611940298507|  1.75724719865717|
|     [150, 480]|     [356]|0.9347826086956522|1.7331835601955863|
|[110, 296, 318]|     [356]|0.8802816901408